#### Hadar Kosto
#### 208847129

In [32]:
import requests
import json
import pandas as pd

# Define the API endpoints and parameters
distance_url = 'https://maps.googleapis.com/maps/api/distancematrix/json'
distance_params = {
    'units': 'metric',
    'origins': 'Tel Aviv',
    'key': ''
}

geocode_url = 'https://maps.googleapis.com/maps/api/geocode/json'
geocode_params = {
    'key': ''
}

# Read the list of targets from a file
with open('dests.txt') as f:
    targets = f.read().splitlines()

# Define a list to store the results
results = []

# Loop over the targets and retrieve the distance, duration, and geocode for each dest.
for target in targets:
    # Retrieve the distance and duration
    distance_params['destinations'] = target
    try:
        response = requests.get(distance_url, params=distance_params)
        if not response.status_code == 200:
            print("HTTP error",response.status_code)
        else:
            try:
                response_data = response.json()
                if not response_data["status"] == "OK":
                    print("error in google API")
                else: 
                    distance = response_data['rows'][0]['elements'][0]['distance']['value'] / 1000 
                    duration = response_data['rows'][0]['elements'][0]['duration']['text']
            except:
                print("Response not in valid JSON format")
    except:
        print("Something went wrong with requests.get")
    
    
    # Retrieve the geocode
    geocode_params['address'] = target
    try:
        response = requests.get(geocode_url, params=geocode_params)
        if not response.status_code == 200:
            print("HTTP error",response.status_code)
        else:
            try:
                data = response.json()
                if not data["status"] == "OK":
                    print("error in google API")
                else: 
                    location = data['results'][0]['geometry']['location']
                    latitude = location['lat']
                    longitude = location['lng']
            except:
                print("Response not in valid JSON format")
    except:
        print("Something went wrong with requests.get")
    
    
    # Add the results to the list
    results.append((target, distance, duration, longitude, latitude))

# Create a pandas DataFrame from the results
df = pd.DataFrame(results, columns=['Target', 'Distance_km', 'Duration', 'Longitude', 'Latitude'])


In [33]:
# Print the DataFrame
print(df)

      Target  Distance_km          Duration  Longitude   Latitude
0   Istanbul     1815.227   21 hours 3 mins  28.978359  41.008238
1  Amsterdam     4532.541    2 days 0 hours   4.904139  52.367573
2   Valletta     3792.883    2 days 3 hours  14.514100  35.899237
3      Basel     4093.629    1 day 20 hours   7.588576  47.559599
4       Doha     2164.477  22 hours 39 mins  51.531040  25.285447


In [21]:
# Sort the DataFrame by the Distance_km column in descending order
df_sorted = df.sort_values('Distance_km', ascending=False)

# Select the first 3 rows
df_furthest = df_sorted.head(3)

# Print the results
print(df_furthest)


      Target  Distance_km        Duration  Longitude   Latitude
1  Amsterdam     4532.619  2 days 0 hours   4.904139  52.367573
3      Basel     4092.812  1 day 20 hours   7.588576  47.559599
2   Valletta     3792.883  2 days 3 hours  14.514100  35.899237
